## Clustering des GHM de la base de données hospitalisation

### 1. Utilisation de *sentence_transformers* pour créer des clusters de GHM par similarité sémantique.

In [1]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
import csv
import time
import random as rd

In [2]:
from sentence_transformers import SentenceTransformer, util

# Sentence embeddings model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [16]:
data=pd.read_csv("base_full_hospit_anonyme.csv")

C:\Users\trist\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
GHM=data["GRG_GHM"].drop_duplicates().sort_values().reset_index(drop=True)
GHM.head()

0    01C01S
1    01C031
2    01C032
3    01C033
4    01C034
Name: GRG_GHM, dtype: object

In [107]:
GHM.to_csv("../data/GHM.csv", index=False)

In [46]:
corpus_sentences=GHM
type(corpus_sentences)

pandas.core.series.Series

In [47]:
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus. This might take a while


Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [48]:
corpus_embeddings.shape

torch.Size([2446, 384])

In [90]:
clusters = util.community_detection(corpus_embeddings,
                                    min_community_size=9,
                                    threshold=0.8,
                                   )

In [91]:
nb_clusters=len(clusters)
df_cluster=pd.DataFrame(columns=['Cluster','GHM'])
j=0

for i, cluster in enumerate(clusters):
    print(i+1, cluster)
    cluster_len=len(cluster)
  
    for sentence_id in cluster[0:len(cluster)]:
        df_cluster.loc[j,['Cluster']]=i+1
        df_cluster.loc[j,['GHM']]=corpus_sentences[sentence_id]
        j=j+1

1 [3, 4, 9, 14, 25, 26, 197, 198, 203, 205, 210, 215, 216, 217, 221, 222, 225, 226, 228, 230, 234, 235, 264, 265, 266, 269, 270, 271, 272, 273, 274, 275, 276, 277, 279, 280, 281, 282, 286, 287, 297, 299, 314, 315, 316, 318, 320, 336, 337, 338, 350, 365, 499, 500, 501, 504, 506, 507, 512, 526, 527, 528, 529, 532, 533, 537, 539, 549, 556, 558, 560, 562, 566, 568, 574, 580, 589, 590, 591, 763, 765, 766, 768, 770, 771, 774, 775, 776, 777, 779, 780, 781, 782, 784, 785, 786, 789, 790, 791, 792, 797, 799, 804, 805, 806, 808, 809, 810, 811, 815, 816, 817, 818, 819, 821, 836, 837, 838, 840, 842, 843, 844, 845, 847, 852, 863, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 968, 969, 970, 971, 972, 973, 974, 975, 983, 984, 985, 986, 992, 994, 995, 1002, 1012, 1013, 1014, 1015, 1024, 1027, 1028, 1029, 1030, 1036, 1037, 1038, 1041, 1042, 1044, 1047, 1048, 1049, 1051, 1056, 1057, 1058, 1060, 1061, 1068, 1069, 1070, 1071, 1075, 1076, 

22 [1602, 1603, 1611, 1612, 1616, 1617, 1618, 1619, 1622, 1624, 1628, 1629, 1630, 1631, 1636, 1637, 1638, 1643, 1644, 1645, 1646, 1733]
23 [32, 33, 301, 1062, 1063, 1484, 1595, 1765, 1770, 1773, 1774, 1776, 1777, 1780, 1781, 1782, 1783, 1784, 1785, 1788, 1794]
24 [51, 89, 103, 116, 128, 244, 245, 313, 386, 402, 409, 416, 436, 449, 456, 468, 710, 714, 725, 735, 741]
25 [22, 219, 513, 525, 535, 554, 609, 795, 802, 813, 823, 824, 941, 1045, 1052, 1066, 1073, 1360, 1365, 1372, 1393]
26 [233, 283, 288, 292, 541, 553, 576, 830, 854, 1116, 1123, 1128, 1131, 1143, 1147, 1150, 1197, 2163, 2167, 2178]
27 [1034, 1247, 1266, 1524, 1532, 1540, 1542, 1547, 1660, 1670, 1890, 1895, 1899, 1900, 1901, 1903, 1904, 1905, 2112]
28 [1537, 1539, 1674, 1800, 1821, 2053, 2056, 2135, 2137, 2139, 2140, 2141, 2142, 2146, 2149, 2151, 2156, 2158, 2253]
29 [71, 78, 93, 96, 258, 719, 903, 982, 987, 996, 1004, 1415, 1416, 1494, 1495, 2187, 2233, 2234]
30 [60, 247, 690, 697, 704, 772, 875, 882, 889, 915, 988, 997, 1005

In [88]:
for i in range(1,len(clusters)+1):
    print("Cluster", i, ":", df_cluster[df_cluster['Cluster']==i]['GHM'].tolist())

Cluster 1 : ['07C082', '07C083', '08C023', '08C043', '08C062', '08C063', '08C123', '08C132', '08C141', '08C142', '08C212', '08C213', '08C214', '08C223', '08C243', '08C253', '08C273', '08C274', '08C323', '08C331', '08C332', '08C341', '08C351', '08C352', '08C353', '08C354', '08C371', '08C372', '08C373', '08C382', '08C391', '08C392', '08C393', '08C394', '08C401', '08C402', '08C411', '08C412', '08C421', '08C422', '08C423', '08C424', '08C431', '08C432', '08C434', '08C442', '08C443', '08C451', '08C452', '08C461', '08C462', '08C463', '08C464', '08C472', '08C473', '08C474', '08C481', '08C482', '08C483', '08C484', '08C491', '08C492', '08C493', '08C494', '08C501', '08C502', '08C503', '08C504', '08C513', '08C522', '08C523', '08C524', '08C531', '08C532', '08C533', '08C541', '08C542', '08C543', '08C544', '08C551', '08C552', '08C561', '08C562', '08C563', '08C564', '08C581', '08C582', '08C614', '08C623', '08C624']
Cluster 2 : ['05C214', '06C031', '06C033', '06C034', '06C041', '06C042', '06C043', '06C

In [92]:
df_cluster

,Cluster,GHM
0,1,01C033
1,1,01C034
2,1,01C044
3,1,01C053
4,1,01C082
...,...,...
2336,87,08M19V
2337,87,11M12V
2338,87,12M02V
2339,87,12M03V


### 2. Clustering des GHM par CMD (Catégorie majeure de diagnostics)

In [102]:
pd.read_csv("../data/CMD_GHM.csv", index_col=0)

,CMD
Code,
1,Affections du système nerveux
2,Affections de l'œil
3,"Affections des oreilles, du nez, de la gorge, ..."
4,Affections de l'appareil respiratoire
5,Affections de l'appareil circulatoire
6,Affections du tube digestif
7,Affections du système hépatobiliaire et du pan...
8,Affections et traumatismes de l'appareil muscu...
9,"Affections de la peau, des tissus souscutanés ..."


In [124]:
clusters_CMD=[]
for i in GHM:
    clusters_CMD.append(i[:2])

In [125]:
df_clusters_CMD=pd.DataFrame()
df_cluster_CMD['Cluster']=clusters_CMD
df_cluster_CMD['GHM']=GHM

In [126]:
df_cluster_CMD

,Cluster,GHM
0,01,01C01S
1,01,01C031
2,01,01C032
3,01,01C033
4,01,01C034
...,...,...
2441,28,28Z23Z
2442,28,28Z24Z
2443,28,28Z25Z
2444,90,90H02Z


In [ ]:
df_cluster_CMD.to_csv("clusters_CMD")